In [203]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from sklearn.metrics import silhouette_samples, silhouette_score

In [260]:
!wget https://raw.githubusercontent.com/rajdeepmondal1/customer-segmentation/main/50000%20Sales%20Records.csv

--2022-04-18 07:44:26--  https://raw.githubusercontent.com/rajdeepmondal1/customer-segmentation/main/50000%20Sales%20Records.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6240886 (6.0M) [text/plain]
Saving to: ‘50000 Sales Records.csv.1’

50000 Sales Records 100%[===================>]   5.95M  --.-KB/s    in 0.08s   

2022-04-18 07:44:26 (79.1 MB/s) - ‘50000 Sales Records.csv.1’ saved [6240886/6240886]



In [261]:
new_dataset = pd.read_csv('/content/50000 Sales Records.csv')

In [262]:
new_dataset

,Region,Country,Item Type,Sales Channel,Order Priority,Order Date,Order ID,Ship Date,Units Sold,Unit Price,Unit Cost,Total Revenue,Total Cost,Total Profit
0,Sub-Saharan Africa,Namibia,Household,Offline,M,8/31/2015,897751939,10/12/2015,3604,668.27,502.54,2408445.08,1811154.16,597290.92
1,Europe,Iceland,Baby Food,Online,H,11/20/2010,599480426,1/9/2011,8435,255.28,159.42,2153286.80,1344707.70,808579.10
2,Europe,Russia,Meat,Online,L,6/22/2017,538911855,6/25/2017,4848,421.89,364.69,2045322.72,1768017.12,277305.60
3,Europe,Moldova,Meat,Online,L,2/28/2012,459845054,3/20/2012,7225,421.89,364.69,3048155.25,2634885.25,413270.00
4,Europe,Malta,Cereal,Online,M,8/12/2010,626391351,9/13/2010,1975,205.70,117.11,406257.50,231292.25,174965.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,Central America and the Caribbean,Dominican Republic,Baby Food,Offline,C,2/12/2015,350891578,3/15/2015,5470,255.28,159.42,1396381.60,872027.40,524354.20
49996,Central America and the Caribbean,Cuba,Office Supplies,Online,M,3/28/2013,748260629,5/3/2013,5803,651.21,524.96,3778971.63,3046342.88,732628.75
49997,Asia,Vietnam,Personal Care,Offline,L,11/11/2016,322932231,12/18/2016,1678,81.73,56.67,137142.94,95092.26,42050.68
49998,Sub-Saharan Africa,Sierra Leone,Clothes,Online,M,8/28/2010,492142713,9/21/2010,4820,109.28,35.84,526729.60,172748.80,353980.80


--2022-04-18 07:44:01--  https://raw.githubusercontent.com/rajdeepmondal1/customer-segmentation/main/50000%20Sales%20Records.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6240886 (6.0M) [text/plain]
Saving to: ‘50000 Sales Records.csv’

50000 Sales Records 100%[===================>]   5.95M  --.-KB/s    in 0.08s   

2022-04-18 07:44:01 (78.7 MB/s) - ‘50000 Sales Records.csv’ saved [6240886/6240886]

